In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df=pd.read_csv('../data/raw/fake_real_news.csv')

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44898 entries, 0 to 44897
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    44898 non-null  object
 1   text     44898 non-null  object
 2   subject  44898 non-null  object
 3   date     44898 non-null  object
 4   target   44898 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 1.7+ MB


In [3]:
df2 = pd.read_csv('../data/raw/fake_or_real_news.csv')

In [6]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6335 entries, 0 to 6334
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  6335 non-null   int64 
 1   title       6335 non-null   object
 2   text        6335 non-null   object
 3   label       6335 non-null   object
dtypes: int64(1), object(3)
memory usage: 198.1+ KB


In [8]:
df2.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
df3 = pd.read_csv('../data/raw/fake_train.csv')

In [7]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20800 entries, 0 to 20799
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      20800 non-null  int64 
 1   title   20242 non-null  object
 2   author  18843 non-null  object
 3   text    20761 non-null  object
 4   label   20800 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 812.6+ KB


In [9]:
# Columnas actuales: title, text, subject, date, target
# Acción: Renombrar 'target' a 'label'. (Aquí ya 1 es Fake, así que no cambiamos valores)
df_clean = df[['title', 'text', 'target']].copy()
df_clean = df_clean.rename(columns={'target': 'label'})

In [10]:
# Acción: La columna label dice 'FAKE'/'REAL'. Hay que mapearla a 1/0.
df2_clean = df2[['title', 'text', 'label']].copy()
label_mapping = {'FAKE': 1, 'REAL': 0} 
df2_clean['label'] = df2_clean['label'].map(label_mapping)

In [11]:
# Columnas actuales: id, title, author, text, label
# Acción: Seleccionar columnas. (Aquí ya 1 es Fake, no cambiamos valores)
df3_clean = df3[['title', 'text', 'label']].copy()

In [12]:
# Concatenamos los tres dataframes limpios
df_final = pd.concat([df_clean, df2_clean, df3_clean], ignore_index=True)

In [14]:
# 1. VERIFICAR: ¿Cuántos títulos duplicados hay actualmente?
duplicados = df_final['title'].duplicated().sum()
print(f"Cantidad de títulos duplicados encontrados: {duplicados}")

Cantidad de títulos duplicados encontrados: 9786


In [15]:
# 2. Pre-limpieza importante:
# A veces el mismo título tiene espacios extra al final en un dataset y en otro no.
# Convertimos a string para evitar errores con nulos y quitamos espacios.
df_final['title'] = df_final['title'].astype(str).str.strip()

In [16]:
# keep='first' -> Mantiene la primera aparición y borra las siguientes.
# subset=['title'] -> Solo mira la columna 'title' para decidir si es duplicado.
df_final = df_final.drop_duplicates(subset=['title'], keep='first')

In [17]:
# 4. TRATAR NULOS (Importante para el Dataset 3)
# El Dataset 3 tenía algunos títulos nulos. Al convertir a string arriba quedaron como "nan".
# Es mejor borrarlos si no tienen título.
df_final = df_final[df_final['title'] != 'nan']

In [18]:
# 5. VERIFICACIÓN FINAL
nuevos_duplicados = df_final['title'].duplicated().sum()
total_filas = len(df_final)

In [19]:
print("-" * 30)
print(f"Duplicados restantes: {nuevos_duplicados}")
print(f"Total de noticias únicas finales: {total_filas}")

------------------------------
Duplicados restantes: 0
Total de noticias únicas finales: 62238


In [20]:
# Es recomendable mezclar los datos (shuffle) para que no queden ordenados por dataset
df_final = df_final.sample(frac=1).reset_index(drop=True)

In [21]:
# Eliminar filas vacías si las hay (a veces el texto viene nulo)
df_final = df_final.dropna(subset=['text'])

In [22]:
conteo = df_final['label'].value_counts()
porcentaje = df_final['label'].value_counts(normalize=True) * 100

print("--- Distribución Numérica ---")
print(f"Total de noticias Falsas (1): {conteo.get(1, 0)}")
print(f"Total de noticias Verdaderas (0): {conteo.get(0, 0)}")
print("\n--- Porcentajes ---")
print(porcentaje)

--- Distribución Numérica ---
Total de noticias Falsas (1): 27844
Total de noticias Verdaderas (0): 34355

--- Porcentajes ---
label
0    55.234007
1    44.765993
Name: proportion, dtype: float64


In [24]:
df_final = df_final[['title', 'text', 'label']]

(fake:1, Real: 0)

In [ ]:
#df_final.to_csv('../data/raw/data.csv', index=False)

In [ ]:
# Crear una columna temporal con el conteo de palabras
df_final['word_count'] = df_final['text'].apply(lambda x: len(str(x).split()))

# Ver estadísticas básicas
print(df_final['word_count'].describe())

# Graficar la distribución
plt.figure(figsize=(10, 6))
plt.hist(df_final['word_count'], bins=50, color='skyblue', edgecolor='black')
plt.title('Distribución de la cantidad de palabras por noticia')
plt.xlabel('Número de palabras')
plt.ylabel('Cantidad de noticias')
plt.show()